In [1]:
from spatialSSL.Dataloader import FullImageDatasetConstructor
from spatialSSL.Utils import split_dataset
from spatialSSL.Training import train, train_classification
from spatialSSL.Models import *
from spatialSSL.Training import train_epoch
from spatialSSL.Testing import test
from spatialSSL.Dataset import InMemoryGraphDataset
import numpy as np

/home/cheng-wei_liao/miniconda3/envs/master_prak/lib/python3.11/site-packages/geopandas/_compat.py:124: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
/home/cheng-wei_liao/miniconda3/envs/master_prak/lib/python3.11/site-packages/spatialdata/__init__.py:9: UserWarning: Geopandas was set to use PyGEOS, changing to shapely 2.0 with:

	geopandas.options.use_pygeos = True

If you intended to use PyGEOS, set the option to False.
  _check_geopandas_using_shapely()
/home/cheng-wei_liao/miniconda3/envs/master_prak/lib/python3.11/site-packages/torch_geometric/typing.py:31: UserWarning: An issue occurred while importing 'torch-scatter'. Disabling its usage. Stacktrace: /home/cheng-wei_liao/miniconda3/envs/master_prak/lib/python3.11/site-packages/torch_scatter/_version_cpu.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKNSt7__cxx1112basic

In [2]:
import torch
import zipfile

# Define a function to load the data from the ZIP file
def load_from_zip(zip_path, file_name):
    with zipfile.ZipFile(zip_path, 'r') as zipf:
        with zipf.open(file_name) as file:
            return torch.load(file)

# Load the pre_train_list and pre_val_list from the ZIP file
pre_train_list = load_from_zip('./processed_data/pre_training_data_img6_r30_n1_random_01.zip', 'pre_train_list.pt')
pre_val_list = load_from_zip('./processed_data/pre_training_data_img6_r30_n1_random_01.zip', 'pre_val_list.pt')

# Now pre_train_list and pre_val_list contain the loaded data


In [3]:
from torch_geometric.loader import DataLoader

# Create DataLoader objects for pre-training and pre-validation
pre_train_loader = DataLoader(pre_train_list, batch_size=1, shuffle=True)
pre_val_loader = DataLoader(pre_val_list, batch_size=1, shuffle=False)

# Now you can use pre_train_loader and pre_val_loader in your training loop


In [4]:
device

NameError: name 'device' is not defined

In [ ]:
# Run pretraining

in_channels  = 550
hidden_channels_1 = 256
hidden_channels_2 = 33
out_channels= 550

# Pretraining
# Define the device
device =  torch.device('cuda' if torch.cuda.is_available() else 'cpu') #"cpu"

# Create the model
model = GAT4(in_channels, hidden_channels_1,hidden_channels_2, out_channels).to(device) # in_channels is set to 100 as an example. Please replace it with your actual feature size.

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 300
patience = 8

train(model=model, train_loader=pre_train_loader, val_loader=pre_val_loader, criterion=criterion, num_epochs= num_epochs, patience = patience, optimizer= optimizer,weight_loss = False, model_path="./models/img6_r30_n1_random_01_GAT4_0.001.pt'")

In [1]:
model

NameError: name 'model' is not defined

In [5]:
from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss

# Run pretraining

in_channels  = 550
hidden_channels = 256
num_classes= 33

# Pretraining
# Define the device
device =  torch.device('cuda' if torch.cuda.is_available() else 'cpu') #"cpu"

model = GATClassification(in_channels, hidden_channels, num_classes).to(device)
criterion = CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

train_classification(model, pre_train_loader, pre_val_loader, criterion, num_classes=num_classes,optimizer=optimizer)

Epoch 1/100, train loss: 2.2422, train accuracy: 0.3258, train auprc: 0.0468, val loss: 1.2544, val accuracy: 0.5647, val auprc: 0.5647, Time: 2.0494s
Epoch 2/100, train loss: 1.2812, train accuracy: 0.4463, train auprc: 0.0624, val loss: 1.0718, val accuracy: 0.6041, val auprc: 0.6041, Time: 1.4357s
Epoch 3/100, train loss: 1.1654, train accuracy: 0.4936, train auprc: 0.0731, val loss: 1.1113, val accuracy: 0.6096, val auprc: 0.6096, Time: 1.4271s
Epoch 4/100, train loss: 1.0951, train accuracy: 0.5211, train auprc: 0.0789, val loss: 0.9783, val accuracy: 0.6185, val auprc: 0.6185, Time: 1.8316s
Epoch 5/100, train loss: 1.0324, train accuracy: 0.5408, train auprc: 0.0827, val loss: 0.9935, val accuracy: 0.6223, val auprc: 0.6223, Time: 1.3749s
Epoch 6/100, train loss: 1.0041, train accuracy: 0.5546, train auprc: 0.0848, val loss: 0.9245, val accuracy: 0.6287, val auprc: 0.6287, Time: 1.4058s
Epoch 7/100, train loss: 0.9576, train accuracy: 0.5661, train auprc: 0.0864, val loss: 0.9309